In [1]:
%pip install --upgrade langchain huggingface-hub sentence_transformers -q
%pip install unstructured -q
%pip install unstructured[local-inference] -q
%pip install tiktoken openai -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [18]:
#load documents using langchain document loader
from langchain.document_loaders import DirectoryLoader

# I compiled all the required documents in one folder
directory = "directory locaton of your folder"

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents=load_docs(directory)

#make chunks from the pdf
from langchain.text_splitter import RecursiveCharacterTextSplitter
#create chunks of 1000 words each
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100, separators=["\n\n","\n","(?<=\.)", " ",""])
docs = text_splitter.split_documents(documents)
print(len(docs))

In [ ]:
docs[0]

In [9]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
#use openAI embedder
import os
import openai
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv

os.environ["OPENAI_API_KEY"] =  'your_openai_api_key'
Embedding = OpenAIEmbeddings()

#check size of embeddings, if using specific vector database
query_result = Embedding.embed_query("Hello World")
len(query_result)

1536

In [12]:
#load generated emebeddings into Azure Blob Storage
#connection details from Azure where the pickle files are stored and updated.

import pickle
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from dotenv import load_dotenv
dotenv_path = 'C:\\.env'  # Replace with your actual file path 
load_dotenv(dotenv_path)

#connection details from Azure where the pickle files are stored and updated.
connection_string = os.getenv('AZURE_CONNECTION_STRING')
container_name = os.getenv('AZURE_CONTAINER_NAME')

blob_service_client = BlobServiceClient.from_connection_string(connection_string)




def save_to_blob(data, blob_name):
    serialized_data = pickle.dumps(data)
    blob_client = blob_service_client.get_blob_client(container_name,blob_name)
    blob_client.upload_blob(serialized_data, overwrite=True)


def load_from_blob(blob_name):
    blob_client = blob_service_client.get_blob_client(container_name, blob_name)
    serialized_data = blob_client.download_blob().readall()
    data = pickle.loads(serialized_data)
    return data

In [13]:
#generate and save embeddings to azure blob storage using FAISS vector store
import faiss
from langchain.vectorstores import FAISS
embeddings = OpenAIEmbeddings()

db = FAISS.from_documents(docs,embeddings)

In [14]:
faiss.write_index(db.index, "docs.index")
db.index=None

In [15]:
save_to_blob(db, 'embeddings.pkl')
save_to_blob('docs.index', 'docs.pkl')

In [ ]:
#test, load from blob
db = load_from_blob('embeddings.pkl')
k1 = load_from_blob('docs.pkl')

index = faiss.read_index(k1)
db.index = index